<a href="https://colab.research.google.com/github/cras-lab/Finance/blob/main/SamSung_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

필요한 모듈 임포트

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from sklearn.model_selection import train_test_split

삼성전자 주가 중 종가(Close)열 부분만 가져온다.

In [ ]:
df = yf.download("005935.KS", start="2020-01-01", end="2022-2-15", progress=False).Close

훈련과 테스트 분할

In [ ]:
y_train, y_test = train_test_split( df, shuffle=False  )

X_train = np.arange(0, len(y_train) )
X_test = np.arange( len(y_train), len(df))
n_features = 1


y_train = np.array( y_train )
y_test = np.array( y_test )

train_series = y_train.reshape( (len(y_train), n_features))
test_series = y_test.reshape( (len(y_test), n_features))

그래프 표시(훈련부분과 테스트 부분을 분리해서 표시)

In [ ]:
fig, ax = plt.subplots( 1,1, figsize=(15,4))
ax.plot( X_train, y_train, lw=3, label="train data")
ax.plot(X_test, y_test, lw=3, label="test data")
ax.legend( loc="lower left")
plt.show()

주가를 0과 1사이의 값으로 표준화(Normalization)한 다음 그래프 표시

In [ ]:
max_price = np.max( train_series )
min_price = np.min( train_series )

train_series = (train_series - min_price) / (max_price-min_price)
test_series = (test_series - min_price) / (max_price-min_price)

fig, ax = plt.subplots( 1,1, figsize=(15,4))
ax.plot( X_train, train_series, lw=3, label="train data")
ax.plot(X_test, test_series, lw=3, label="test data")
ax.legend( loc="lower left")
plt.show()


TimeseriesGenerator 형태로 변환

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
look_back = 20

train_generator = TimeseriesGenerator( train_series, train_series,
                                      length=look_back,
                                      batch_size = 10)


test_generator = TimeseriesGenerator(test_series, test_series,
                                     length= look_back,
                                     batch_size=10)

LSTM 네트워크 구성

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM

# 결과가 좋지 않으면, 여기로 돌아와서 뉴런 개수를 변경하면서 테스트 해보자(4 ==> 12 ==> 48)
n_neurons = 4

model = Sequential()
model.add( LSTM( n_neurons, input_shape=(look_back, n_features)))
model.add( Dense(1))
model.compile( optimizer='adam', loss='mse')

모델을 적합화 한 다음, 예측 생성

In [ ]:
model.fit( train_generator, epochs=50, verbose=0)

test_prediction = model.predict( test_generator )

예측을 그래프로 표시

In [ ]:
x= np.arange( len(df) - len(test_prediction ), len(df) )
fig, ax= plt.subplots( 1,1, figsize=(15,5))
ax.plot( X_test, y_test, lw=3, c='y', label='test_data')
ax.plot( x, test_prediction*(max_price-min_price)+min_price,\
        lw=3, c='r', linestyle=':', label='predictions')
ax.legend( loc='lower left')
plt.show()


결과가 좋지 않다면 앞으로 돌아가서 뉴런개수를 12,48으로 늘려가면서 변화가 있는지 관찰해 보자.